In [ ]:
! pip install requests

In [2]:
import requests

In [46]:
base_url = 'https://pokeapi.co/api/v2/'

def get_pokemon(name):
    url = f'{base_url}/pokemon/{name}'
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return f'Failed to retrieve data: {response.status_code}'

pokemon_name = 'ditto'
pokemon_data = get_pokemon(pokemon_name)
print(f"Name: {pokemon_data['name']} "
      f"\nId: {pokemon_data['id']}"
      f"\nWeight: {pokemon_data['weight']}kg",
      f"\nAbilities: {pokemon_data['moves'][0]['move']['name']}")


Name: ditto 
Id: 132
Weight: 40kg 
Abilities: transform
